In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

countryRoad = r'suc2cExudTip/none_1476_{}_10to14_20mn_0s_10/'


In [3]:
def getValT(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

exceptPset = []
exceptPsetTime = []
for i in range(99):
    # path2file = pathresults + 'param99none'+str(i)+'0dx_2dumux_10c_10.0to11.0_20mn_0s_192_100/'
    path2file = countryRoad.format(i)#'newMucil/none_1476_'+str(i)+'_10to14_20mn_0s_10/'
    #'none_55_'+str(i)+'_10to11_20mn_0s_5/'# 'fixeUpdatenone'+str(i)+'0dx_2dumux_10c_10.0to25.0_20mn_0s_192_100/'
    tt = getValT(path2file, "time.txt",names= ['time','Qlight'])
    time = tt.iloc[:,0]
    if max(time) < 14: 
        exceptPset.append(i)
        exceptPsetTime.append(max(time))
        print(i, (int((max(time)-10.)*24*10)/10), end ="h, ")
print('\n',len(exceptPset))#, (int((min(exceptPsetTime)-10.)*24*10)/10))


 0


In [4]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path+"cyl_val/", "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist
# cylinder max konz (per cell)
def getData_(cid,gId,path2file):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

exceptPset =[]
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    for pSet in allPSets:
        print(pSet,end =", ")

        #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        path2file =  countryRoad.format(pSet)#'newMucil/none_1476_'+str(i)+'_10to14_20mn_0s_10/'
        
        rr = getCylIdx(path2file)
        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        for cid in range(nKonz,volIdx):
            nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits
        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]
        
        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", dtype =int)
        id_roots = id_orgs[np.where(ot_orgs==2)]
        
        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])
        
        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))
        
        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])
            
            
        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens

    return GiniAll #cid pSet rr
    

In [5]:
GiniAll = getData()

0, ( 439 ), 1, ( 432 ), 2, ( 414 ), 3, ( 449 ), 4, ( 508 ), 5, ( 404 ), 6, ( 408 ), 7, ( 531 ), 8, ( 419 ), 9, ( 452 ), 10, ( 432 ), 11, ( 407 ), 12, ( 402 ), 13, ( 424 ), 14, ( 419 ), 15, ( 468 ), 16, ( 454 ), 17, ( 402 ), 18, ( 451 ), 19, ( 480 ), 20, ( 410 ), 21, ( 402 ), 22, ( 449 ), 23, ( 478 ), 24, ( 500 ), 25, ( 423 ), 26, ( 445 ), 27, ( 479 ), 28, ( 419 ), 29, ( 450 ), 30, ( 445 ), 31, ( 432 ), 32, ( 420 ), 33, ( 437 ), 34, ( 478 ), 35, ( 492 ), 36, ( 463 ), 37, ( 474 ), 38, ( 447 ), 39, ( 430 ), 40, ( 403 ), 41, ( 493 ), 42, ( 426 ), 43, ( 507 ), 44, ( 414 ), 45, ( 480 ), 46, ( 482 ), 47, ( 527 ), 48, ( 432 ), 49, ( 506 ), 50, ( 500 ), 51, ( 528 ), 52, ( 476 ), 53, ( 410 ), 54, ( 432 ), 55, ( 515 ), 56, ( 503 ), 57, ( 402 ), 58, ( 508 ), 59, ( 442 ), 60, ( 538 ), 61, ( 421 ), 62, ( 528 ), 63, ( 449 ), 64, ( 495 ), 65, ( 430 ), 66, ( 471 ), 67, ( 424 ), 68, ( 424 ), 69, ( 461 ), 70, ( 544 ), 71, ( 445 ), 72, ( 508 ), 73, ( 532 ), 74, ( 395 ), 75, ( 476 ), 76, ( 452 ), 77, ( 432

In [131]:
#assert np.array(GiniAll).shape == (nToGet, numPset, 232) # some have a couple of segments more

/tmp/ipykernel_1126/1273250247.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert np.array(GiniAll).shape == (nToGet, numPset, 232) #all same shape


AssertionError: 

In [19]:
np.array(GiniAll).shape

/tmp/ipykernel_324427/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 99)

In [6]:
# get param set
paramsets =  pd.read_csv('../../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
paramsets['k_C,S'] /=paramsets['theta']
paramsets['k_O,S'] /=paramsets['theta']
paramsets.drop(labels='theta', inplace=True, axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca',
                   'ccd','css2','co2','yX','yY','yZ','vol','lenSeg',
                   'st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + paramsets.columns.tolist()+['pSet']
df = pd.DataFrame(columns=column_names)

for pset in allPSets:#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_to_duplicate_df = pd.DataFrame(paramsets.iloc[pset]).T
    duplicated_rows_df = rows_to_duplicate_df.loc[np.repeat(rows_to_duplicate_df.index.values, N)]
    # Duplicate the specific rows N times
    #print('init dupl row shape',duplicated_rows_df.shape)
    for jj, outVn in enumerate(outputvalsname):
        duplicated_rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    duplicated_rows_df['pSet'] = pset
    
    path2file =countryRoad.format(pset)#'newMucil/none_1476_'+str(pset)+'_10to11_20mn_0s_10/'
    simTime = get_last_line_as_numpy_array(path2file+"totalComputetime.txt", dtype =float)[0]
    
    duplicated_rows_df['simTime'] = simTime
    # Concatenate the original DataFrame with the duplicated specific rows
    df = pd.concat([df, duplicated_rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [7]:
df.to_pickle('./cyl10to11_4d.pkl')

In [8]:
df

,wat,cs,cl,coa,cod,cca,ccd,css2,co2,yX,...,Y_C,Y_O,CSS_max,k_sorp,CL_init,CS_init,pSet,simTime,co,cc
0,0.043164,3.580200e-07,0.000910,1.559599e-09,9.273342e-06,3.710699e-08,9.274678e-06,0.000008,4.204311e-06,-0.298434,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,27061.792136,0.000009,0.000009
1,0.007359,3.545819e-07,0.000863,1.547891e-09,9.273352e-06,3.708371e-08,9.274679e-06,0.000008,4.203686e-06,0.043922,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,27061.792136,0.000009,0.000009
2,0.008891,4.191260e-07,0.000798,1.798020e-09,9.273140e-06,3.753666e-08,9.274561e-06,0.000010,4.213155e-06,-0.447197,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,27061.792136,0.000009,0.000009
3,0.008445,3.552012e-07,0.000800,1.549898e-09,9.273350e-06,3.708737e-08,9.274678e-06,0.000008,4.203759e-06,-0.104340,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,27061.792136,0.000009,0.000009
4,0.007028,3.021884e-05,0.000931,1.103364e-05,8.488178e-09,1.211594e-05,4.814225e-07,0.000593,9.941887e-05,-0.125120,...,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,27061.792136,0.000011,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45856,0.107388,7.993337e-07,0.000675,2.991717e-07,8.972346e-06,3.850334e-07,9.129081e-06,0.000014,1.624442e-05,0.882095,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,11931.685178,0.000009,0.000010
45857,0.014028,3.608405e-06,0.000675,7.763518e-07,8.505577e-06,1.123396e-06,8.898858e-06,0.000051,3.979705e-05,-0.281210,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,11931.685178,0.000009,0.000010
45858,0.014119,2.021994e-06,0.000675,6.971693e-07,8.584745e-06,1.109735e-06,8.919545e-06,0.000048,4.012103e-05,0.457628,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,11931.685178,0.000009,0.000010
45859,0.215349,2.123169e-07,0.000675,1.458284e-07,9.120594e-06,1.665676e-08,9.160251e-06,0.000005,6.511811e-07,0.721308,...,0.021234,0.092764,0.082080,0.116598,8.059087,0.002712,98,11931.685178,0.000009,0.000009
